# 处理缺失值(Handling Missing Data)
大多数教程里使用的数据与现实工作中的数据的区别在于后者很少是干净整齐的，许多目前流行的数据集都会有数据缺失的现象。更为甚者，处理不同数据源缺失值的方法还不同。

我们将在本节介绍一些处理缺失值的通用规则，Pandas 对缺失值的表现形式，并演示Pandas 自带的几个处理缺失值的工具的用法。本节以及全书涉及的缺失值主要有三种形式：null、NaN 或NA。

In [1]:
import numpy as np
import pandas as pd

## Pandas的缺失值(Missing Data in Pandas)
Pandas 最终选择用标签方法表示缺失值，包括两种Python 原有的缺失值：浮点数据类型的NaN 值，以及Python 的None 对象。后面我们将会发现，虽然这么做也会有一些副作用，但是在实际运用中的效果还是不错的。

#### 1. None：Python对象类型的缺失值
Pandas 可以使用的第一种缺失值标签是None，它是一个Python 单体对象，经常在代码中表示缺失值。由于None 是一个Python 对象，所以不能作为任何NumPy / Pandas 数组类型的缺失值，只能用于'object' 数组类型（即由Python 对象构成的数组）：

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

这里dtype=object 表示NumPy 认为由于这个数组是Python 对象构成的，因此将其类型判断为object。虽然这种类型在某些情景中非常有用，对数据的任何操作最终都会在Python 层面完成，但是在进行常见的快速操作时，这种类型比其他原生类型数组要消耗更多的资源：

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %time np.arange(1E7, dtype=dtype).sum()
    print()

dtype = object
Wall time: 539 ms

dtype = int
Wall time: 23 ms



使用Python 对象构成的数组就意味着如果你对一个包含None 的数组进行累计操作，如sum() 或者min()，那么通常会出现类型错误：

    vals1.sum()#会报错

这就是说，在Python 中没有定义整数与None 之间的加法运算。

#### 2. NaN：数值类型的缺失值
另一种缺失值的标签是NaN（全称Not a Number，不是一个数字），是一种按照IEEE 浮点
数标准设计、在任何系统中都兼容的特殊浮点数：

In [4]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

请注意，NumPy 会为这个数组选择一个原生浮点类型，这意味着和之前的object 类型数组不同，这个数组会被编译成C 代码从而实现快速操作。你可以把NaN 看作是一个数据类病毒——它会将与它接触过的数据同化。无论和NaN 进行何种操作，最终结果都是NaN：

In [5]:
1 + np.nan

nan

In [6]:
0 * np.nan

nan

虽然这些累计操作的结果定义是合理的（即不会抛出异常），但是并非总是有效的：

In [7]:
vals2.sum(), vals2.min(), vals2.max()

d:\python37\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)
d:\python37\lib\site-packages\numpy\core\_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


(nan, nan, nan)

NumPy 也提供了一些特殊的累计函数，它们可以忽略缺失值的影响：

In [8]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

**谨记，NaN 是一种特殊的浮点数，不是整数、字符串以及其他数据类型。**

#### 3. Pandas中NaN与None的差异
虽然NaN 与None 各有各的用处，但是Pandas 把它们看成是可以等价交换的，在适当的时候会将两者进行替换：

In [9]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [10]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [11]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

请注意，除了将整型数组的缺失值强制转换为浮点数，Pandas 还会自动将None 转换为NaN。（需要注意的是，现在GitHub 上Pandas 项目中已经有人提议增加一个原生的整型NA，不过到现在还尚未实现。）

尽管这些仿佛会魔法的类型比R 语言等专用统计语言的缺失值要复杂一些，但是Pandas的标签/ 转换方法在实践中的效果非常好，在我个人的使用过程中几乎没有出过问题。Pandas 对NA 缺失值进行强制转换的规则如表 所示。

Pandas对不同类型缺失值的转换规则

| 类型     | 缺失值转换规则 | NA标签值     |
| -------- | -------------- | ------------ |
| floating | No change      | np.nan       |
| object   | No change      | None or np.nan |
| integer  | Cast tofloat64 | np.nan       |
| boolean  | Cast toobject  | None or np.nan |

**需要注意的是，Pandas 中字符串类型的数据通常是用object 类型存储的。**

## 处理缺失值(Operating on Null Values)
我们已经知道，Pandas 基本上把None 和NaN 看成是可以等价交换的缺失值形式。为了完成这种交换过程，Pandas 提供了一些方法来发现、剔除、替换数据结构中的缺失值，主要包括以下几种。

* isnull()创建一个布尔类型的掩码标签缺失值。
* notnull()与isnull() 操作相反。
* dropna()返回一个剔除缺失值的数据。
* fillna()返回一个填充了缺失值的数据副本。

本节将用简单的示例演示这些方法。

#### 1. 发现缺失值

Pandas 数据结构有两种有效的方法可以发现缺失值：isnull() 和notnull()。每种方法都返回布尔类型的掩码数据，例如：

In [12]:
data = pd.Series([1, np.nan, 'hello', None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [13]:
type(data.isnull())

pandas.core.series.Series

就像前面介绍的，布尔类型掩码数组可以直接作为Series 或DataFrame 的索引使用：

In [14]:
data[data.notnull()]

0        1
2    hello
dtype: object

在Series 里使用的isnull() 和notnull() 同样适用于DataFrame，产生的结果同样是布尔类型。

#### 2. 剔除缺失值
除了前面介绍的掩码方法，还有两种很好用的缺失值处理方法，分别是dropna()（剔除缺失值）和fillna()（填充缺失值）。在Series 上使用这些方法非常简单：

In [15]:
data.dropna()

0        1
2    hello
dtype: object

In [16]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

而在DataFrame 上使用它们时需要设置一些参数，例如下面的DataFrame：

In [17]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


我们没法从DataFrame 中单独剔除一个值，要么是剔除缺失值所在的整行，要么是整列。根据实际需求，有时你需要剔除整行，有时可能是整列，DataFrame 中的dropna() 会有一些参数可以配置。

默认情况下，dropna() 会剔除任何包含缺失值的整行数据：

In [18]:
df.dropna()

,0,1,2
1,2.0,3.0,5


可以设置按不同的坐标轴剔除缺失值，比如axis=1（或axis='columns'）会剔除任何包含缺失值的整列数据：

In [19]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [20]:
df.dropna(axis=1)

,2
0,2
1,5
2,6


但是这么做也会把非缺失值一并剔除，因为可能有时候只需要剔除全部是缺失值的行或列，或者绝大多数是缺失值的行或列。这些需求可以通过设置how 或thresh 参数来满足，它们可以设置剔除行或列缺失值的数量阈值。

默认设置是how='any'，也就是说只要有缺失值就剔除整行或整列（通过axis 设置坐标轴）。你还可以设置how='all'，这样就只会剔除全部是缺失值的行或列了：

In [21]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [22]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


还可以通过thresh 参数设置行或列中非缺失值的最小数量，从而实现更加个性化的配置：

In [23]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


第1 行与第3 行被剔除了，因为它们只包含两个非缺失值。

#### 3. 填充缺失值
有时候你可能并不想移除缺失值，而是想把它们替换成有效的数值。有效的值可能是像0、1、2 那样单独的值，也可能是经过填充（imputation）或转换（interpolation）得到的。虽然你可以通过isnull() 方法建立掩码来填充缺失值，但是Pandas 为此专门提供了一个fillna() 方法，它将返回填充了缺失值后的数组副本。

来用下面的Series 演示：

In [24]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

我们将用一个单独的值来填充缺失值，例如用0：

In [25]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

可以用缺失值前面的有效值来从前往后填充（forward-fill）：

In [26]:
# 从前往后填充
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

也可以用缺失值后面的有效值来从后往前填充（back-fill）：

In [27]:
# 从后往前填充
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

DataFrame 的操作方法与Series 类似，只是在填充时需要设置坐标轴参数axis：

In [28]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [29]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


In [30]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


需要注意的是，假如在从前往后填充时，需要填充的缺失值前面没有值，那么它就仍然是
缺失值。